In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
%matplotlib inline

In [2]:
donations_13to17=pd.read_pickle('donations_13to17.pkl')

In [3]:
donations_13to17.head()

,id,first_gift_year,lapsed_count,recovered_count,max_consec_giving_years,total_giving_years,count_2013,count_2014,count_2015,count_2016,...,behavior_2017,gainloss_2013,gainloss_2014,gainloss_2015,gainloss_2016,gainloss_2017,gainloss_2014_amt,gainloss_2015_amt,gainloss_2016_amt,gainloss_2017_amt
0,1f4b5b6e68445c6c4a0509b3aca93f38,2015-01-01,4,4,4,4,0,0,42,64,...,upgrade,none,none,gain,gain,gain,0.0,18211.38,21873.77,22559.83
1,4aaab6d244bf3599682239ed5591af8a,2016-01-01,2,1,1,1,0,0,0,1,...,lapse_new,none,none,none,gain,loss,0.0,0.00,25.00,-25.00
2,0b0765dc9c759adc48a07688ba25e94e,2015-01-01,3,2,2,2,0,0,2,2,...,lapse_repeat,none,none,gain,gain,loss,0.0,20.00,20.00,-40.00
3,377944ad61f72d800b25ec1862aec363,2016-01-01,2,1,1,1,0,0,0,1,...,lapse_new,none,none,none,gain,loss,0.0,0.00,25.00,-25.00
4,6d5b22d39e68c656071a842732c63a0c,2015-01-01,4,3,2,3,0,0,1,1,...,lapse_repeat,none,none,gain,loss,loss,0.0,70.00,-45.00,-25.00


In [5]:
donations_13to17.amount_2013.describe()

count    1.876468e+06
mean     1.620692e+01
std      4.297376e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.427944e+05
Name: amount_2013, dtype: float64

In [18]:
donations_13to17.query('amount_2013 > 0').describe()['amount_2013']

count    320069.000000
mean         95.016273
std        1036.920844
min           0.400000
25%          20.000000
50%          25.000000
75%          75.000000
max      442794.380000
Name: amount_2013, dtype: float64

In [23]:
donations_13to17.query('amount_2014 > 0').describe()['amount_2014']

count    382803.000000
mean        108.259304
std        1169.334035
min           1.000000
25%          25.000000
50%          50.000000
75%         100.000000
max      570161.900000
Name: amount_2014, dtype: float64

In [24]:
donations_13to17.query('amount_2015 > 0').describe()['amount_2015']

count    446037.000000
mean        113.174654
std        1013.841004
min           0.920000
25%          25.000000
50%          50.000000
75%         100.000000
max      346566.480000
Name: amount_2015, dtype: float64

In [25]:
donations_13to17.query('amount_2016 > 0').describe()['amount_2016']

count    555836.000000
mean        112.707242
std        1028.602129
min           0.060000
25%          25.000000
50%          50.000000
75%         100.000000
max      294946.630000
Name: amount_2016, dtype: float64

In [26]:
donations_13to17.query('amount_2017 > 0').describe()['amount_2017']

count    574591.000000
mean        126.817661
std        1628.855404
min           1.000000
25%          25.000000
50%          50.000000
75%         100.000000
max      634709.970000
Name: amount_2017, dtype: float64

In [8]:
behaviors_by_number=donations_13to17[['id', 'first_gift_year', 'behavior_2013','behavior_2014', 'behavior_2015', 'behavior_2016', 'behavior_2017']].copy()

In [9]:
donor_behavior=pd.melt(behaviors_by_number, 
        id_vars =['id', 'first_gift_year'], 
        value_vars =['behavior_2013','behavior_2014', 'behavior_2015', 'behavior_2016', 'behavior_2017'], 
        var_name='year', 
        value_name='behavior') 

In [10]:
donor_behavior1 = donor_behavior.query('behavior != "none"')

In [11]:
donor_behavior1.loc[:,'year'] = np.where((donor_behavior1.year == 'behavior_2013'),'2013',donor_behavior1.year)
donor_behavior1.loc[:,'year'] = np.where((donor_behavior1.year == 'behavior_2014'),'2014',donor_behavior1.year)
donor_behavior1.loc[:,'year'] = np.where((donor_behavior1.year == 'behavior_2015'),'2015',donor_behavior1.year)
donor_behavior1.loc[:,'year'] = np.where((donor_behavior1.year == 'behavior_2016'),'2016',donor_behavior1.year)
donor_behavior1.loc[:,'year'] = np.where((donor_behavior1.year == 'behavior_2017'),'2017',donor_behavior1.year)

C:\Users\erinm\Anaconda3\lib\site-packages\pandas\core\indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [12]:
conditions = [(donor_behavior1.loc[:,'behavior'] == 'new') |              
              (donor_behavior1.loc[:,'behavior'] == 'recover'),
              (donor_behavior1.loc[:,'behavior'] == 'upgrade') |
              (donor_behavior1.loc[:,'behavior'] == 'downgrade') |
              (donor_behavior1.loc[:,'behavior'] == 'maintain'),
              (donor_behavior1.loc[:,'behavior'] == 'lapse_new') |
              (donor_behavior1.loc[:,'behavior'] == 'lapse_repeat')]
              
choices = ['gain', 'retain', 'loss']

donor_behavior1.loc[:,'g/l'] = np.select(conditions, choices, default='none')

C:\Users\erinm\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\erinm\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
donor_behavior1

,id,first_gift_year,year,behavior,g/l
6,8a1875762c85932fff192ea126ccdff2,2013-01-01,2013,new,gain
7,a3f070e439d52de72ca62dc41f9b16a4,2013-01-01,2013,new,gain
8,bd323208dc78b1c74b62664b768f3176,2013-01-01,2013,new,gain
9,6dd6113f89f2766d3b0707ef2a46260c,2013-01-01,2013,new,gain
10,391f14831940fc7bc41df2cd7fb06030,2013-01-01,2013,new,gain
...,...,...,...,...,...
9382335,e6a4ef71c2ad1fae992b0ca093b41dad,2016-01-01,2017,lapse_new,loss
9382336,17349045834fff2e3a1644a240ada6a5,2016-01-01,2017,lapse_new,loss
9382337,07cbfd22c88371d0fc4e5d68799dc4cc,2016-01-01,2017,lapse_new,loss
9382338,90449de3dfeeac866d2ec5c88ac9da60,2016-01-01,2017,lapse_new,loss


In [253]:
donor_behavior1.to_pickle('donor_behavior1.pkl')

### START HERE

In [6]:
donor_behavior1=pd.read_pickle('donor_behavior1.pkl')

In [7]:
donor_behavior1.head()

,id,first_gift_year,year,behavior,g/l
6,8a1875762c85932fff192ea126ccdff2,2013-01-01,2013,new,gain
7,a3f070e439d52de72ca62dc41f9b16a4,2013-01-01,2013,new,gain
8,bd323208dc78b1c74b62664b768f3176,2013-01-01,2013,new,gain
9,6dd6113f89f2766d3b0707ef2a46260c,2013-01-01,2013,new,gain
10,391f14831940fc7bc41df2cd7fb06030,2013-01-01,2013,new,gain


In [14]:
donor_behavior_grouped=donor_behavior1.groupby(['year', 'g/l','behavior','first_gift_year']).count()['id'].reset_index()

In [15]:
conditions = [(donor_behavior_grouped.loc[:,'g/l'] == 'gain'),
              (donor_behavior_grouped.loc[:,'g/l'] == 'retain'),
              (donor_behavior_grouped.loc[:,'g/l'] == 'loss')]
              
choices = [donor_behavior_grouped.id, donor_behavior_grouped.id, donor_behavior_grouped.id*-1]

donor_behavior_grouped.loc[:,'count'] = np.select(conditions, choices, default='none')

In [16]:
donor_behavior_grouped.head()

,year,g/l,behavior,first_gift_year,id,count
0,2013,gain,new,2013-01-01,320069,320069
1,2014,gain,new,2014-01-01,320673,320673
2,2014,loss,lapse_new,2013-01-01,257939,-257939
3,2014,retain,downgrade,2013-01-01,23847,23847
4,2014,retain,maintain,2013-01-01,11171,11171


In [92]:
fig = px.bar(donor_behavior_grouped, x="behavior1", y="count", color="behavior",
             title="Long-Form Input", 
             facet_col="year1", 
#             facet_col="behavior1", 
             category_orders={"year": [2013,2014,2015,2016,2017], "behavior1": ["gain", "retain","loss"]})
fig.show()

In [182]:
grouped13=donor_behavior_grouped.query('year1 == "2013"')
grouped14=donor_behavior_grouped.query('year1 == "2014"')
grouped15=donor_behavior_grouped.query('year1 == "2015"')
grouped16=donor_behavior_grouped.query('year1 == "2016"')
grouped17=donor_behavior_grouped.query('year1 == "2017"')

In [183]:
grouped14

,year1,behavior1,behavior,first_gift_year,id,count
1,2014,gain,new,2014-01-01,320673,320673
2,2014,loss,lapse_new,2013-01-01,257939,-257939
3,2014,retain,downgrade,2013-01-01,23847,23847
4,2014,retain,maintain,2013-01-01,11171,11171
5,2014,retain,upgrade,2013-01-01,27112,27112


In [186]:
total13=grouped13.id.sum()
total14=grouped14.id.sum()
total15=grouped15.id.sum()
total16=grouped16.id.sum()
total17=grouped17.id.sum()

In [187]:
total17

1030736

In [141]:
g14=grouped14.groupby(['behavior1', 'behavior']).sum().reset_index()

In [142]:
g14

,behavior1,behavior,id
0,gain,new,320673
1,loss,lapse_new,257939
2,retain,downgrade,23847
3,retain,maintain,11171
4,retain,upgrade,27112


In [164]:
fig = make_subplots(rows=2, cols=2, start_cell="bottom-left")


fig.add_trace(go.Bar(x=grouped14['behavior1'], y=grouped14['count']), 
                     row=1, col=1),

fig.add_trace(go.Bar(x=grouped15['behavior1'], y=grouped15['count']),
              row=1, col=2)

fig.add_trace(go.Bar(x=grouped16['behavior1'], y=grouped16['count']),
              row=2, col=1)

fig.add_trace(go.Bar(x=grouped17['behavior1'], y=grouped17['count']),
              row=2, col=2)

fig.show()

In [160]:
fig = go.Figure()
for contestant, group in grouped14.groupby('behavior1'):
    fig.add_trace(go.Bar(x=group["behavior1"], y=group["id"], name=contestant))
    #  hovertemplate="Contestant=%s<br>Fruit=%%{x}<br>Number Eaten=%%{y}<extra></extra>"% contestant))
#fig.update_layout(legend_title_text = "Contestant")
#fig.update_xaxes(title_text="Fruit")
#fig.update_yaxes(title_text="Number Eaten")
fig.show()

In [61]:
fig = px.bar(donor_behavior_grouped, x="behavior1", y="behavior", color="year1", barmode="group", title="Long-Form Input")
fig.show()

In [ ]:
df = donor_behavior_grouped
for i, row in enumerate(df["Date"]):
    p = re.compile(" 00:00:00")
    datetime = p.split(df["Date"][i])[0]
    df.iloc[i, 1] = datetime

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Mining-revenue-USD"],
        mode="lines",
        name="mining revenue"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Hash-rate"],
        mode="lines",
        name="hash-rate-TH/s"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Bitcoin mining stats for 180 days",
)

fig.show()

In [ ]:
fig.add_trace(
    go.Table(
        header=dict(
            values=["Year", "behavior1", "behavior",
                    "count"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Bitcoin mining stats for 180 days",
)

fig.show()

In [190]:
donor_behavior_grouped.head()

,year1,behavior1,behavior,first_gift_year,id,count
0,2013,gain,new,2013-01-01,320069,320069
1,2014,gain,new,2014-01-01,320673,320673
2,2014,loss,lapse_new,2013-01-01,257939,-257939
3,2014,retain,downgrade,2013-01-01,23847,23847
4,2014,retain,maintain,2013-01-01,11171,11171


In [193]:
conditions = [donor_behavior_grouped.loc[:,'year1'] == '2013',
              donor_behavior_grouped.loc[:,'year1'] == '2014',
              donor_behavior_grouped.loc[:,'year1'] == '2015',
              donor_behavior_grouped.loc[:,'year1'] == '2016',
              donor_behavior_grouped.loc[:,'year1'] == '2017']
              
choices = [(donor_behavior_grouped['id']/total13*100), (donor_behavior_grouped['id']/total14*100), (donor_behavior_grouped['id']/total15*100), (donor_behavior_grouped['id']/total16*100), (donor_behavior_grouped['id']/total17*100)]

donor_behavior_grouped.loc[:,'perc'] = np.select(conditions, choices, default='none')

In [194]:
df = donor_behavior_grouped

fig = go.Figure(data=[go.Table(
    header=dict(values=('year', 'g/l','sub-g/l','count','perc'),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df.year1, df.behavior1, df.behavior, df.id, df.perc],
               fill_color='lavender',
               align='left'))
])

fig.show()